In [1]:
import importlib
import pandas as pd
import numpy as np
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
""" 
Для формування списку ЄДРПОУ навчальних закладів, які потрібно наносити на мапу, використані бази даних МОН, опубліковані у листопаді 2019 року на Порталі відкритих даних.

### Перелік закладів загальної середньої та дошкільної освіти (ЗЗСДО)
    https://data.gov.ua/dataset/bd70757a-ab49-46be-a650-608a9b3bd7b1
    
### Перелік закладів позашкільної освіти (державна форма власності)
    https://data.gov.ua/dataset/9a88fb96-a043-4769-acf1-1c4a948ec285
    Перелік закладів позашкільної освіти системи Міністерства освіти і науки України
    https://data.gov.ua/dataset/18d7fb70-3781-4017-88d2-651e14d14211
    Заклади професійної (професійно-технічної) освіти
    https://data.gov.ua/dataset/824513a9-aa5e-45c8-b45e-dced6909761a
"""

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [3]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns
print("Дані з файлу викачки BIPro Prozorro: " + "df_raw" + str(df_raw.shape))

Дані з файлу викачки BIPro Prozorro: df_raw(537893, 11)


In [129]:
# EXPERIMENTS

expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';', low_memory = False)
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';', low_memory = False)

In [104]:
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])
print("expdata_sc", expdata_sc.shape)

expdata_sc (15206, 21)


In [146]:
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])
print("expdata_dnz", expdata_dnz.shape)

expdata_dnz (15745, 21)


In [108]:
filter_upravlinnya_osvity = ["департамент.*освіт", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_raw[df_raw['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_upravlinnya_osvity)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()
print("Кількість унікальних ЄДРПОУ управлінь освіти, edrpou_upravlin_osvity_prozorro", len(edrpou_upravlin_osvity))

Кількість унікальних ЄДРПОУ департаментів управлінь edrpou_upravlin_osvity_prozorro 58


In [153]:
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)
print(df_mon_data.shape)

In [163]:
df_mon_data.sample(1)

,university_name,edrpou,university_type_name,education_type_name,university_level,location_type,university_financing_type_name,post_index,koatuu_id,language,...,post_index_u,koatuu_id_u,region_name_u,koatuu_name_u,university_address_u,university_phone,university_email,university_site,university_director_fio,koatuu_name
10558,"Опорний заклад загальної середньої освіти ""Пов...",20141398.0,заклад середньої освіти,загальносвітня школа,I-III,СЕЛО,комунальна,45050,722187401,! not applicable,...,45050,722187401,Волинська область,Поворськ,"вул. Шевченка, 12",(03352)95537,schoolpovorsk@ukr.net,https://povorsk-litsey.e-schools.info,! not applicable,Поворськ


In [168]:
df_mon_data.pivot_table(index=['university_name'], aggfunc='size')
df_mon_data.to_csv('df_not_school_tenders.csv');

In [113]:
expdata_sc_upravlin_osvity = expdata_sc[expdata_sc['edrpou'].isin(edrpou_upravlin_osvity)]
edrpou_upravlin_osvity_mon = expdata_sc_upravlin_osvity['edrpou'].unique().tolist()
print("Кількість унікальних ЄДРПОУ управлінь освіти, edrpou_upravlin_osvity_mon", len(edrpou_upravlin_osvity_mon))

Кількість унікальних ЄДРПОУ департаментів управлінь edrpou_upravlin_osvity_mon 11


In [119]:
mask = (expdata_sc['edrpou'].isin(edrpou_upravlin_osvity_prozorro)) | (expdata_sc['edrpou'].isin(edrpou_upravlin_osvity_mon))
expdata_sc_only_schools = expdata_sc.loc[~mask]
expdata_sc_only_schools.shape

(15169, 21)

##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro. Виділення закупівель шкіл-замовників , згідно з базою МОН (пошук по колонці "IDOrganizator"

In [120]:
mon_data = get_mon_data()

df_tenders_by_schools_by_edrpou = df_raw[df_raw['IDOrganizator'].isin(mon_data)]
df_undefined_1 = df_raw[~df_raw['IDOrganizator'].isin(mon_data)]

print("Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou " + str(df_tenders_by_schools_by_edrpou.shape))
print("Невизначені записи: df_undefined_1 " + str(df_undefined_1.shape))

Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou (38817, 11)
Невизначені записи: df_undefined_1 (499076, 11)


In [122]:
len(mon_data)

29794

In [ ]:
# evaluation of results of Step 1

""" 

TODO:

    викреслити зі списку Бази МОН всі управління, департаменти, відділи освіти

    прибрати з результатів закупівлі, де в полі Организатор є такі слова:
        - департамент
        - управління освіти
        - відділ освіти
        
* результати фільтрування у файлі "filter_results/df_tenders_by_schools_by_edrpou" 

"""

# df_tenders_by_schools_by_edrpou['Организатор'].value_counts()

In [7]:
# possible solution for Step 1

filter_keywords_step_1_exclude = ["департамент", "управління", "відділ", "міжшкільний"]

df_tenders_by_schools_by_edrpou_2 = df_tenders_by_schools_by_edrpou[~df_tenders_by_schools_by_edrpou['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_1_exclude)), 
            case = False, na = False, regex = True)]
print("Додатково відфільтрована база закупівель за базою МОН: df_tenders_by_schools_by_edrpou_2", df_tenders_by_schools_by_edrpou_2.shape)

filter_keywords_step_1_include = ["департамент", "управління", "відділ"]

df_undefined_2 = df_tenders_by_schools_by_edrpou[df_tenders_by_schools_by_edrpou['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_1_include)), 
            case = False, na = False, regex = True)]
print("Невизначені записи: df_undefined_2", df_undefined_2.shape)

df_tenders_by_schools_by_edrpou_2 (27125, 11)
df_undefined_2 (11687, 11)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


##### Step 1.5: Об'єднання двох датафреймів, де не визначені школи, в один

In [9]:
df_undefined_3 = pandas.concat([df_undefined_1, df_undefined_2])
print("Невизначені записи: df_undefined_3", df_undefined_3.shape[0])

df_undefined_3 510763


##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [12]:
df_tenders_by_schools_by_keywords = df_undefined_3[df_undefined_3['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)]
df_undefined_4 = df_undefined_3[~df_undefined_3['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)]

print("Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords " + str(df_tenders_by_schools_by_keywords.shape))
print("Невизначені записи: df_undefined_4 " + str(df_undefined_4.shape))

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords (13459, 11)
Невизначені записи: df_undefined_4 (497304, 11)


In [ ]:
# evalutaion of results of Step 2

""" 

TODO:
    додати в filter_keywords регулярні вирази для витягування в полі "Организатор" закупівель за ключовим словом: 
        - "школа І-ІІІ ступен" OR "школа І-ІІ ступен"
        - "ліцей" OR "гімназія" AND NOT "професійн")
        - "(ясла-садок)"
        - "Навчально-виховний комплекс"
        
* результати фільтрування у файлі "filter_results/df_tenders_by_schools_by_keywords" 

"""

# df_tenders_by_schools_by_keywords['Организатор'].value_counts()

In [18]:
# possible solution for Step 2

filter_keywords_step_2_include = ["школа І-ІІІ? ступен", "садок", "навчально.виховн", "ліцей", "гімназі"]
filter_keywords_step_2_exclude = ["професій"]

df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords[df_tenders_by_schools_by_keywords['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_include)), 
            case = False, na = False, regex = True)]
print("Додаткове фільтрування датафрейму df_tenders_by_schools_by_keywords", df_tenders_by_schools_by_keywords_2.shape)

df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords_2[~df_tenders_by_schools_by_keywords_2['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_exclude)), 
            case = False, na = False, regex = True)]
print("Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів", df_tenders_by_schools_by_keywords_2.shape)

# df_tenders_by_schools_by_keywords_2['Организатор'].value_counts()

Додаткове фільтрування датафрейму df_tenders_by_schools_by_keywords (6014, 11)
Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів (3718, 11)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # This is added back by InteractiveShellApp.init_path()


##### Step 2.5: Об'єднання двох датафреймів шкіл-замовників в один

In [20]:
df_tenders_by_schools = pandas.concat([df_tenders_by_schools_by_edrpou_2, df_tenders_by_schools_by_keywords_2])

print("Закупівлі шкіл-замовників: df_tenders_by_schools " + str(df_tenders_by_schools.shape))
print("Невизначені записи: df_undefined_4 " + str(df_undefined_4.shape))

Закупівлі шкіл-замовників: df_tenders_by_schools (30843, 11)
Невизначені записи: df_undefined_4 (497304, 11)


##### Step 3: Класифікація шкіл-замовників по типах

In [34]:
columns_to_classify = ['Организатор']

df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, columns_to_classify, filter_priority, filter_name_priority)

In [35]:
# analysis of Step 3

pd.set_option("display.max_colwidth", 200)
for_analysis_step_3 = df_tenders_by_schools_classified[df_tenders_by_schools_classified['Тип закладу'] == 'Невідомо'][['Организатор', 'Тип закладу']].sort_values(by=['Организатор'])
for_analysis_step_3['Организатор'].unique().tolist()

['Галицький коледж імені В’ячеслава Чорновола',
 'МЕЛ№2 ММР МО',
 'Муніципальний заклад вищої освіти "Київська Академія мистецтв"',
 "Позашкільний навчально-виховний заклад Палац дитячої та юнацької творчості Солом'янського району м.Києва",
 'Рівненський міський Палац дітей та молоді',
 'Рівненський навчально-реабілітаційний центр "Особлива дитина" Рівненської міської ради']

##### Step 4: Нумерація шкіл-замовників

In [58]:
columns_to_numerate = ['Организатор']

df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, columns_to_numerate, reg_number)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


##### Step 4.5: Найменування шкіл-замовників

In [65]:
columns_to_name = ['Организатор']

df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, columns_to_name, reg_quotes)

In [68]:
# видалити закупівлі позашкільних закладів через фільтрування filter_poza

print('Було закупівель', df_tenders_by_schools_named.shape)
df_tenders_by_schools_named = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)),
                case = False, na = False, regex = True)]
print('Стало закупівель', df_tenders_by_schools_named.shape)

Було закупівель (30181, 15)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


Стало закупівель (30181, 15)


In [69]:
# analysis of Step 4

# перетворити пусті строки в NaN
df_tenders_by_schools_named['Номер'] = df_tenders_by_schools_named['Номер'].replace(r'^\s*$', np.nan, regex=True)

for_analysis_step_4 = df_tenders_by_schools_named[df_tenders_by_schools_named['Назва'].isnull() & 
                                  df_tenders_by_schools_named['Номер'].isnull()]
print('Навчальні заклади, де не визначились номер чи назва:')
for_analysis_step_4['Организатор'].unique().tolist()

Навчальні заклади, де не визначились номер чи назва:


['Авіакосмічний ліцей ім. І.Сікорського НАУ м. Києва',
 'Галицький коледж імені В’ячеслава Чорновола',
 'Харківська гімназія 116 Харківської міської ради Харківської області',
 'Перша міська гімназія Черкаської міської ради Черкаської області',
 'Івано-Франківський природничо-математичний ліцей Івано-Франківської міської ради Івано-Франківської області',
 'КОМУНАЛЬНА УСТАНОВА СУМСЬКА КЛАСИЧНА ГІМНАЗІЯ СУМСЬКОЇ МІСЬКОЇ РАДИ',
 'Львівська правнича гімназія',
 'КУ Хмельницький колегіум імені Володимира Козубняка',
 'Тернопільська спеціалізована загальноосвітня школа І ступеня повного дня з поглибленим вивченням іноземних мов Тернопільської міської ради Тернопільської області',
 'Рівненський міський Палац дітей та молоді',
 'Політехнічниий ліцей Національного технічного університету України КПІ м. Києва',
 'ЧЕРНІВЕЦЬКИЙ ВІЙСЬКОВО-СПОРТИВНИЙ ЛІЦЕЙ-ІНТЕРНАТ ЧЕРНІВЕЦЬКОЇ МІСЬКОЇ РАДИ',
 'Класична гімназія Ужгородської міської ради Закарпатської області',
 'КЗ "ХАРКІВСЬКИЙ УНІВЕРСИТЕТСЬКИЙ ЛІЦ

In [ ]:
# evalutaion of results of Step 4

""" 

TODO:
   ??? ще не зрозуміло

"""

##### Step 5: Класифікація шкіл-НЕ-замовників

In [ ]:
##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [ ]:
step_mark = 'step0'
action_mark = 'check'
number_0 = '0'
columns_to_check_step0 = ['Организатор']

# Перший спосіб відібрати навчальні заклади, які є замовниками: пошук за ЄДРПОУ в полі "IDOrganizator"
df_tenders_by_schools_by_edrpou = filter_dataframe(df_raw, positive_edrpous(), ['IDOrganizator'])

# Створити базу для закупівель навчальних закладів, які НЕ є замовниками
df_merged_left = pandas.merge(df_raw, df_tenders_by_schools_by_edrpou, on='ИдентификаторЛота', 
                              how = 'left', indicator = True, suffixes = ('','_temp'))
df_undefined_0 = df_merged_left[df_merged_left._merge=='left_only']
df_undefined_0 = df_undefined_0[training_data_columns]

# Другий спосіб відібрати навчальні заклади, які є замовниками: пошук за ключовими словами в полі "Организатор"
df_step0 = df_undefined_0.copy()

for column in columns_to_check_step0:
    df_step0 = step_check(df_step0, column, number_0)
    
df_tenders_by_schools_by_keywords_0, df_undefined_0 = separate_positives_and_negatives_ogranizator(df_step0, step_mark, action_mark)
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords:", df_tenders_by_schools_by_keywords_0.shape[0])

#### Concatenating 2 dataframes: df_tenders_by_schools_concatenated

In [ ]:
# df_tenders_by_schools_concatenated - 
# Об'єднання двох баз закупівель навчальних закладів, які самостійно купують через ProZorro

frames_tenders_by_schools = [df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_0]
df_tenders_by_schools_concatenated = pandas.concat(frames_tenders_by_schools)
len_df = df_tenders_by_schools_concatenated.shape[0]
df_tenders_by_schools_concatenated.drop_duplicates(subset = 'ИдентификаторЛота', inplace = True)
print("Removed duplicates:", len_df - df_tenders_by_schools_concatenated.shape[0])

### Step 1: 
#### Створити базу закупівель навчальних закладів, які купують у ProZorro через посередників: управління освіти, міністерства

In [ ]:
# TODO Step 1:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "АДЕРСА ПОСТАВКИ"

"""
За результатами точності визначення закупівель вирішити чи фільтрувати за цією колонкою.
Проблема: в полі адреса часто вказують ключові слова, які ніби можуть вказувати на приналежність
закупівлі до шкільних. Наприклад, вул. Шкільна, Гімназійна і т.п.
"""

In [ ]:
step_mark = 'step1'
action_mark = 'check'
number_1 = '1'
columns_to_check_step1 = ['Тендер', 'ОписаниеТендера', 'Лот']
df_step1 = df_undefined_0.copy()

for column in columns_to_check_step1:
    df_step1 = step_check(df_step1, column, number_1)

df_tenders_by_schools_by_keywords_1, df_undefined_1 = separate_positives_and_negatives(df_step1, step_mark, action_mark)
df_tenders_by_schools_by_keywords_1 = df_tenders_by_schools_by_keywords_1[training_data_columns]
df_not_school_tenders = df_undefined_1[training_data_columns]
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords_1:", df_tenders_by_schools_by_keywords_1.shape[0])

In [ ]:
# TODO:
#    Inspect df_not_school_tenders.csv
#   df_not_school_tenders.to_csv('df_not_school_tenders.csv');

### Step 2:
#### Класифікація навчальних закладів, які самостійно купують через ProZorro

In [ ]:
# TODO:
# Визначити пріоритетність фільтрів для визначення категорії. Наприклад, якщо в назві є НВК і дитячий садок.

def classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.contains('|'.join(filter_list), 
                                                                                               case = False, 
                                                                                               na = False)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break;
    for column in columns_to_classify:
        given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

In [ ]:
step_mark = 'step2'
action_mark = 'classification'
columns_to_classify_step2 = ['Организатор']
df_step2 = df_tenders_by_schools_concatenated.copy()

In [ ]:
df_step2_random = df_step2.copy().sample(1000)

#### Filtering

In [ ]:
for idx in range(len(filter_priority)):
    df_step2_random = classification(df_step2_random, columns_to_classify_step2, filter_priority[idx], filter_name_priority[idx])

#### Checking results

In [ ]:
df_step2_random[df_step2_random['Тип закладу'].notnull()].shape

In [ ]:
print(df_step2_random[df_step2_random['Тип закладу'].isnull()].shape)

columns_to_classify_step2_temp = columns_to_classify_step2.copy()
columns_to_classify_step2_temp.append('Тип закладу')

df_step2_random[df_step2_random['Тип закладу'].isnull()][columns_to_classify_step2_temp]

In [ ]:
df_step2_random[df_step2_random['Тип закладу'] == 'ДНЗ'][columns_to_classify_step2_temp].sample(10)

### Step 3:
#### Класифікація навчальних закладів, для яких купують управління освіти, міністерства чи інші органи влади

In [ ]:
# TODO:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "ОРГАНИЗАТОР"

In [ ]:
step_mark = 'step3'
action_mark = 'classification'
columns_to_classify_step3 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step3 = df_tenders_by_schools_by_keywords_1.copy()

In [ ]:
df_step3_random = df_step3.copy().sample(1000)

#### Filtering

In [ ]:
for idx in range(len(filter_priority)):
    df_step3_random = classification(df_step3_random, columns_to_classify_step3, 
                                    filter_priority[idx], filter_name_priority[idx])

#### Checking results

In [ ]:
columns_to_classify_step3_temp = columns_to_classify_step3.copy()
columns_to_classify_step3_temp.append('Тип закладу')

In [ ]:
df_step3_random['Тип закладу'].value_counts()

In [ ]:
df_step3_random[df_step3_random['Тип закладу'].notnull()].shape

In [ ]:
print(df_step3_random[df_step3_random['Тип закладу'].isnull()].shape)
df_step3_random[df_step3_random['Тип закладу'].isnull()][columns_to_classify_step3_temp]

### Step 4:
####  присвоєння навчальним закладам власних назв

In [ ]:
step_mark = 'step4'
action_mark = 'naming'

columns_to_classify_step4_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()

columns_to_classify_step4 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step4 = df_step3.copy()

In [ ]:
def naming(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE).str[0].str[-1]
    return given_df

In [ ]:
df_step4_tenders_by_schools_filtered = naming(df_step4_tenders_by_schools, columns_to_classify_step4_tenders_by_schools, reg_quotes)
df_step4_tenders_by_schools_filtered.sample(10)[['Организатор', 'Организатор_step4_naming']]

In [ ]:
df_step4_filtered = naming(df_step4, columns_to_classify_step4, reg_quotes)
df_step4_filtered.sample(10)

### Step 5:
####  присвоєння номерів

In [ ]:
step_mark = 'step5'
action_mark = 'numeration'

columns_to_classify_step5_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()

columns_to_classify_step5 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step5 = df_step3.copy()

In [ ]:
def numeration(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE)
    return given_df

In [ ]:
df_step5_tenders_by_schools_filtered = numeration(df_step4_tenders_by_schools, columns_to_classify_step4_tenders_by_schools, reg_number)
df_step5_tenders_by_schools_filtered

In [ ]:
df_step5_filtered = numeration(df_step4, columns_to_classify_step4, reg_number)
df_step5_filtered.sample(5)

### Step 6:
####  зведення в єдину базу

### Optional step

#### За ЄДРПОУ видалити рядки із закупівлями закладів, які не є закладами загальної середньої чи дошкільної освіти

In [ ]:
# TODO: 
# До переліку "негативних" ЄДРПОУ включені управління освіти. 
# Тому заклади, де є "управління освіти" в полі організатор, не видаляти

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])['IDOrganizator'].unique().tolist()

In [ ]:
#filter_dataframe_inspect_before_drop_step_0['Организатор'].unique().tolist()

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, filter_poza, ['Организатор'])['Организатор'].unique().tolist()

In [ ]:
#df_step2_positives = filter_dataframe_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])
#df_step2_positives = filter_dataframe_drop(df_step2_positives, filter_poza, ['Организатор'])
#print('Залишилось закупівель навчальних закладів, які не є замовниками:', df_step2_positives.shape[0])